In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']][:1000]

In [5]:
import ast
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [6]:
movies['genres'] = movies['genres'].apply(convert)

In [7]:
movies['keywords'] = movies['keywords'].apply(convert)

In [8]:
def convert3(obj):
    l=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
            counter +=1
        else:
            break
    return l

In [9]:
movies['cast'] = movies['cast'].apply(convert3)

In [10]:
def dirc(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [11]:
movies['crew'] = movies['crew'].apply(dirc)

In [12]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [13]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

In [14]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [15]:
movies = movies[['movie_id', 'title', 'tags']]

In [16]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [17]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [18]:
import nltk

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [20]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [21]:
movies['tags'] = movies['tags'].apply(stem)

## Data Preprocessing Done

# Converting to Vector

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words = 'english')

In [23]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
similarity = cosine_similarity(vectors)

C:\Users\ishwa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [26]:
def recommend(movie):
    movies_index = movies[movies['title'] == 'Avatar'].index[0]
    movies_list = sorted(list(enumerate(similarity[movies_index])), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [27]:
import pickle

In [28]:
pickle.dump(movies, open("movies.pkl", 'wb'))

In [29]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))